In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Input ,concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers
import tensorflow as tf

In [2]:
X_train = load('trainX1.npy')
X_test = load('testX1.npy')
y_train = load('trainY1.npy')
y_test = load('testY1.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [3]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [4]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
n_timesteps

128

In [102]:
model1= Sequential()
model1.add(LSTM(512,input_shape=(n_timesteps,n_features)))
#model1.add(LSTM(256))
#model1.add(Dense(340 ,activation='relu'))
#model1.add(Dropout(0.1))
#model1.add(Dense(100, activation='relu'))
model1.add(Dense(n_outputs, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [103]:
model2= Sequential()
model2.add(Conv1D(filters=32, kernel_size=9, activation='relu',input_shape=(n_timesteps,n_features)))
model2.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model2.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model2.add(Conv1D(filters=128, kernel_size=6, activation='relu'))
model2.add(Dropout(0.5))
#model2.add(MaxPooling1D(pool_size=2))
model2.add(Flatten())
#model2.add(Dense(100, activation='relu'))
model2.add(Dense(n_outputs, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [104]:
input_shape=(n_timesteps,n_features)
commonInput = Input(shape=input_shape)

In [105]:
mergedInput= concatenate([model1(commonInput),model2(commonInput)])
mergedInput.shape

TensorShape([None, 236])

In [106]:
out = Dense(n_outputs, activation='softmax')(mergedInput)
new_model = Model(commonInput,out)

In [107]:
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [108]:
validation_data=(X_validation, y_validation)

In [ ]:
history = model2.fit(X_training, y_training, epochs=10, verbose=True, validation_data=(X_validation, y_validation), batch_size=100)

Epoch 1/10
265/265 [==============================] - 39s 143ms/step - loss: 0.8390 - accuracy: 0.8219 - val_loss: 0.2889 - val_accuracy: 0.9354
Epoch 2/10
265/265 [==============================] - 38s 144ms/step - loss: 0.1753 - accuracy: 0.9558 - val_loss: 0.2155 - val_accuracy: 0.9486
Epoch 3/10
265/265 [==============================] - 38s 143ms/step - loss: 0.0911 - accuracy: 0.9737 - val_loss: 0.1931 - val_accuracy: 0.9550
Epoch 4/10
265/265 [==============================] - 38s 142ms/step - loss: 0.0589 - accuracy: 0.9820 - val_loss: 0.2259 - val_accuracy: 0.9520
Epoch 5/10
265/265 [==============================] - 38s 143ms/step - loss: 0.0453 - accuracy: 0.9857 - val_loss: 0.2002 - val_accuracy: 0.9588
Epoch 6/10
265/265 [==============================] - 38s 143ms/step - loss: 0.0384 - accuracy: 0.9886 - val_loss: 0.2157 - val_accuracy: 0.9579
Epoch 7/10
265/265 [==============================] - 38s 144ms/step - loss: 0.0294 - accuracy: 0.9901 - val_loss: 0.2144 - val_ac

In [101]:
loss, accuracy = model2.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9557
Testing Accuracy: 0.8941
